In [ ]:
import openpyxl
from datetime import datetime, timedelta
import os

class ExcelDemoEntity:
    def __init__(self, column1, column2, column3, column4, column5, column6):
        # Initialize the attributes corresponding to Excel columns
        self.column1 = column1  # Assuming column1 is poiId
        self.column2 = column2  # Assuming column2 is scenicId (commented in initial code)
        self.column3 = column3  # Assuming column3 is one
        self.column4 = column4  # Assuming column4 is two
        self.column5 = column5  # Assuming column5 is three
        self.column6 = column6  # Assuming column6 is florescence

def do_date(date_str, bloom_percent):
    date_parts = date_str.split('-')
    start_date = datetime.strptime(f"2024年{date_parts[0]}", "%Y年%m月%d日")
    end_date = datetime.strptime(f"2024年{date_parts[1]}", "%Y年%m月%d日")
    return (start_date, end_date, bloom_percent)

def main():
    # Path to the Excel file
    file_name = "F:\\工作记录\\2024杜鹃花地图—20240328.xlsx"
    
    # Open the workbook and select the active sheet
    wb = openpyxl.load_workbook(file_name)
    sheet = wb.active
    
    # List to hold SQL statements
    sql_list = []
    
    # Type for 杜鹃花 (Rhododendron)
    plant_type = 3

    # Iterate over the rows in the sheet starting from the third row
    for row in sheet.iter_rows(min_row=3, values_only=True):
        # Create an entity for each row
        entity = ExcelDemoEntity(*row)
        
        # Extract values from the entity
        poi_id = entity.column1
        florescence = entity.column6
        one = entity.column3
        two = entity.column4
        three = entity.column5
        
        # Process the bloom periods
        periods = [
            do_date(one, "0.25"),
            do_date(two, "0.5"),
            do_date(three, "1"),
        ]
        
        # Generate SQL statements for each period
        for start_date, end_date, bloom_percent in periods:
            current_date = start_date
            while current_date <= end_date:
                sql_statement = (
                    f"INSERT INTO \"public\".\"mall_scenic_bloom\"("
                    f"\"poi_id\", \"bloom_percent\", \"start_at\", \"florescence\", \"type\") "
                    f"VALUES ({poi_id}, '{bloom_percent}', '{current_date.date()}', '{florescence}', '{plant_type}');"
                )
                sql_list.append(sql_statement)
                current_date += timedelta(days=1)

    # Write SQL statements to a file
    with open('bloom.sql', 'w', encoding='utf-8') as f:
        for sql in sql_list:
            f.write(sql + '\n')

if __name__ == "__main__":
    main()